In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
import collections
import pandas as pd
from fuzzywuzzy import fuzz, process
import math
pd.set_option('display.max_columns', None)


In [3]:
html = requests.get("https://www.fantasyfootballscout.co.uk/team-news/").text
soup = BeautifulSoup(html, "html.parser")


In [4]:
raw = soup.find_all("div", {"class": re.compile('formation.*')})
teams = [x.text.split('Next')[0] for x in soup.find_all('header')[-20:]]
lineups = [x.text.split('  ')[1:] for x in raw]

In [5]:
xi = {team: lineup for team, lineup in zip(teams, lineups)}

In [6]:
xi

{'Arsenal': ['Ramsdale',
  'Tomiyasu',
  'White',
  'Gabriel Magalhães',
  'Tierney',
  'Partey',
  'Xhaka',
  'Saka',
  'Odegaard',
  'Gabriel Martinelli',
  'Jesus'],
 'Aston Villa': ['Martinez',
  'Cash',
  'Diego Carlos',
  'Mings',
  'Digne',
  'McGinn',
  'Kamara',
  'Jacob Ramsey',
  'Coutinho',
  'Ings',
  'Watkins'],
 'Bournemouth': ['Travers',
  'A Smith',
  'Mepham',
  'Kelly',
  'Zemura',
  'Lerma',
  'L Cook',
  'Billing',
  'Christie',
  'Solanke',
  'Anthony'],
 'Brentford': ['Raya Martin',
  'Hickey',
  'Jansson',
  'Pinnock',
  'Henry',
  'Jensen',
  'Norgaard',
  'Janelt',
  'Mbeumo',
  'Toney',
  'Wissa'],
 'Brighton and Hove Albion': ['Sanchez',
  'Veltman',
  'Dunk',
  'Webster',
  'March',
  'Mwepu',
  'Mac Allister',
  'Moises Caicedo',
  'Cucurella',
  'Welbeck',
  'Trossard'],
 'Chelsea': ['Mendy',
  'Koulibaly',
  'Thiago Silva',
  'Sarr',
  'James',
  'Kovacic',
  'Kanté',
  'Chilwell',
  'Mount',
  'Sterling',
  'Havertz'],
 'Crystal Palace': ['Guaita',
  'C

In [7]:
html = requests.get("https://understat.com/league/EPL").text
soup = BeautifulSoup(html, "html.parser")

In [8]:
pscript = soup.find_all('script')[3].string
sidx = pscript.index("parse") + 7
eidx = pscript.index(");") - 1

In [9]:
playerdata = json.loads(pscript[sidx:eidx].encode('utf8').decode('unicode_escape'))

In [10]:
teamscript = soup.find_all('script')[2].string
sidx = teamscript.index("parse") + 7
eidx = teamscript.index(")") - 1

In [11]:
teamdata = json.loads(teamscript[sidx:eidx].encode('utf8').decode('unicode_escape'))

In [12]:
fd = collections.defaultdict(list)

for d in playerdata:
    for key, value in d.items():
        fd[key].append(value)
players = pd.DataFrame(fd)

In [14]:
h_a

NameError: name 'h_a' is not defined

In [13]:
teamsxg = pd.DataFrame(teamdata.values())
teamsxg = teamsxg.explode("history")
h = teamsxg.pop("history")
teamsxg = pd.concat([teamsxg.reset_index(drop=True), pd.DataFrame(h.tolist())], axis=1)
teamsxg = teamsxg.infer_objects()
teamsxg = teamsxg.groupby(['title', 'h_a']).sum().reset_index()
teamsxg['matches'] = teamsxg['wins'] + teamsxg['draws'] + teamsxg['loses']
teamsxg = teamsxg.rename(columns = {'title':'team','scored':'goals_scored','missed':'goals_conceded'})
teamsxg['goals_conceded_90'] = teamsxg['goals_conceded']/teamsxg['matches']
teamsxg['xG_90'] = teamsxg['xG']/teamsxg['matches']
teamsxg['xGA_90'] = teamsxg['xGA']/teamsxg['matches']
aag = teamsxg.loc[teamsxg['h_a'] == 'a','xG_90'].mean()
ahg = teamsxg.loc[teamsxg['h_a'] == 'h','xG_90'].mean()
aaga = teamsxg.loc[teamsxg['h_a'] == 'a','xGA_90'].mean()
ahga = teamsxg.loc[teamsxg['h_a'] == 'h','xGA_90'].mean()
# teamsxg['xG_diff'] = teamsxg['xG_90'] - teamsxg['xGA_90']
teamsxg['xG_strength'] = teamsxg.apply(lambda row: xgstrength(row['h_a'], row['xG_90'], aag, ahg), axis = 1)
teamsxg['xGA_strength'] = teamsxg.apply(lambda row: xgstrength(row['h_a'], row['xGA_90'], aaga, ahga), axis = 1)

KeyError: 'h_a'

In [8]:
raw

[<div class="formation formation-4-2-3-1"><ul class="row-1"><li title="Ramsdale (Aaron)"> <span class="team-disc ars-dark"></span> <span class="player-name">Ramsdale</span></li></ul><ul class="row-2"><li title="Tomiyasu (Takehiro)"> <span class="team-disc ars-dark"></span> <span class="player-name">Tomiyasu</span></li><li title="White (Ben)"> <span class="team-disc ars-dark"></span> <span class="player-name">White</span></li><li title=""> <span class="team-disc ars-dark"></span> <span class="player-name">Gabriel Magalhães</span></li><li title="Tierney (Kieran)"> <span class="team-disc ars-dark"></span> <span class="player-name">Tierney</span></li></ul><ul class="row-3"><li title="Partey (Thomas)"> <span class="team-disc ars-dark"></span> <span class="player-name">Partey</span></li><li title="Xhaka (Granit)"> <span class="team-disc ars-dark"></span> <span class="player-name">Xhaka</span></li></ul><ul class="row-4"><li title="Saka (Bukayo)"> <span class="team-disc ars-dark"></span> <span

In [614]:
g = teamsxg.groupby(['team'], as_index=False).mean()
teams_xg_90 = dict(zip(g['team'],g['xG_90']))
teamsxg['xG_multiplier'] = teamsxg.apply(lambda row: row['xG_90']/teams_xg_90[row['team']], axis = 1)

In [316]:
test = {"Arsenal":1.27, "Aston Villa":1.63}

In [319]:
for i in range(0,5):
    mt = []
    odd1 = poisspdf(i, 1.63)
    for j in range(0,5):
        mt.append(poisspdf(j, 1.27) * odd1)
    print(mt)

[0.055023220056407245, 0.06987948947163719, 0.044373475814489624, 0.018784771428133944, 0.005964164928432527]
[0.0896878486919438, 0.11390356783876862, 0.07232876557761808, 0.030619177427858326, 0.009721588833345018]
[0.0730955966839342, 0.09283140778859643, 0.05894794394575874, 0.024954629603704537, 0.00792309489917619]
[0.0397152741982709, 0.050438398231804044, 0.032028382877195574, 0.01355868208467946, 0.004304881561885729]
[0.016183974235795394, 0.020553647279460147, 0.013051566022457195, 0.00552516294950688, 0.0017542392364684343]


In [336]:
def calcmult(xg_90, team):
    return xg_90 / teams_xg_90[team]

In [280]:
def xgstrength(h_a, xg_90, aa, ah):
    if h_a == 'a':
        return xg_90/aa
    else:
        return xg_90/ah
    

In [304]:
def calcgoals(h_a, col1, col2, aah, ahg):
    if h_a == 'a':
        return col1 * col2 * aah
    else:
        return col1 * col2 * ahg

In [616]:
players.to_csv('players.csv', index = False)

In [640]:
players = pd.read_csv('players.csv')
players['name_team'] = players['player_name'] + ' ' + players['team_title']
players['xG_90'] = players['xG']/players['time']*90

In [619]:
p = list(filter(lambda x: ',' not in x, sorted(players.team_title.unique())))
x = sorted(list(xi.keys()))
d = {k:v for k, v in zip(x, p)}

In [620]:
lop = [i for j in xi.values() for i in j]

In [598]:
def getname(name):
    highest = process.extractOne(name,list(players.name_team))
    return highest[0]

In [622]:
def adjustxi(xi):
    for team, players in xi.items():
        for idx, player in enumerate(players):
            if player == 'S Armstrong':
                players[idx] = 'Stuart Armstrong'
            elif player == 'D Sanchez':
                players[idx] = 'David Sanchez'
            elif player == 'Wood':
                players[idx] = 'Chris Wood'
            elif player == 'Sanchez':
                players[idx] = 'Robert Sanchez'

In [692]:
players[['position','xG_90']].groupby('position').mean()

,xG_90
position,
D,0.048982
D F M S,0.161876
D M,0.042426
D M S,0.052387
D S,0.040221
F,0.430867
F M,0.319956
F M S,0.243760
F S,0.358740


In [697]:
players[players.position == 'S']

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup,name_team,xG_90
80,484,Divock Origi,3,27,2,0.797817,0,0.000000,5,0,0,0,S,Liverpool,2,0.797817,0.859705,0.061888,Divock Origi Liverpool,2.659389
81,558,Jesse Lingard,9,73,2,0.511964,0,0.071041,4,2,0,0,S,Manchester United,2,0.511964,0.689844,0.133821,Jesse Lingard Manchester United,0.631188
122,8239,Takumi Minamino,8,70,2,1.560086,0,0.491121,3,1,1,0,S,Liverpool,2,1.560086,2.230111,0.178904,Takumi Minamino Liverpool,2.005825
164,1746,Jeff Hendrick,3,65,1,0.017748,0,0.022814,1,1,0,0,S,Newcastle United,1,0.017748,0.160857,0.120296,Jeff Hendrick Newcastle United,0.024574
202,8821,Donny van de Beek,8,59,1,0.741281,0,0.393387,2,3,0,0,S,Manchester United,1,0.741281,1.635464,0.555413,Donny van de Beek Manchester United,1.130767
222,462,Dan Gosling,1,19,0,0.000000,0,0.000000,0,0,0,0,S,Watford,0,0.000000,0.000000,0.000000,Dan Gosling Watford,0.000000
232,574,Troy Deeney,2,19,0,0.109417,0,0.033171,2,1,0,0,S,Watford,0,0.109417,0.142588,0.000000,Troy Deeney Watford,0.518289
258,743,Dwight Gayle,4,16,0,0.060420,0,0.010893,1,1,0,0,S,Newcastle United,0,0.060420,0.071313,0.010893,Dwight Gayle Newcastle United,0.339864
285,987,Joseph Gomez,4,14,0,0.000000,0,0.000000,0,0,0,0,S,Liverpool,0,0.000000,0.000000,0.000000,Joseph Gomez Liverpool,0.000000
289,1053,Keinan Davis,1,10,0,0.000000,0,0.000000,0,0,0,0,S,Aston Villa,0,0.000000,0.028242,0.028242,Keinan Davis Aston Villa,0.000000


In [ ]:
players[players[]]

In [686]:
def getxG(name):
    plname = getname(name)
    shots = players.loc[players['name_team'] == plname, "shots"].values[0]
    minutes = players.loc[players['name_team'] == plname, "time"].values[0]
    if shots > 7 or minutes > 180:
        return players.loc[players['name_team'] == plname, "xG_90"].values[0]
    else:
        if 

In [648]:
# for team in d.keys():
#     for i in xi[team]:
#         print(f"{i} {getname(i + ' ' + d[team])} {team}")

In [687]:
for i in xi['Manchester City']:
    print(i, getxG(i + 'Manchester City'))

Ederson 0.0
Walker 0.010766490126506918
Rúben Dias 0.07876392515366755
Laporte 0.02821966526126432
João Cancelo 0.10556957046267681
De Bruyne 0.2216766666254548
Rodri 0.06017339491117992
Bernardo Silva 0.254312092979235
Jesus 0.6096236543373096
Grealish 0.32657985544600193
Foden 0.37942797312662546


In [679]:
getxG('Ramsdale Arsenal')

'Aaron Ramsdale Arsenal'

In [682]:
getxG('Gabriel Arsenal')

0.07485308957351898

In [688]:
namexg = {}
teamxg = {}
for team, plys in xi.items():
    xgsum = 0
    for ply in plys:
        plxg = getxG(ply + ' ' + d[team])
        xgsum = xgsum + plxg
        namexg[ply] = plxg
    teamxg[team] = xgsum

In [689]:
teamxg

{'Arsenal': 1.5906424644122763, 'Aston Villa': 1.0367192715181872, 'Brentford': 1.555792784872493, 'Brighton and Hove Albion': 1.5569590776464555, 'Burnley': 0.9168758361062198, 'Chelsea': 1.7425590162311395, 'Crystal Palace': 1.4103558308519641, 'Everton': 1.6466766893001494, 'Leeds United': 1.3371790070105611, 'Leicester City': 1.7747221005073104, 'Liverpool': 2.8172383179865466, 'Manchester City': 1.9319486526195437, 'Manchester United': 1.7187607603977826, 'Newcastle United': 1.5401989899293334, 'Norwich City': 1.1178861649513268, 'Southampton': 1.3888341502401684, 'Tottenham Hotspur': 1.4545498592331567, 'Watford': 1.5077940601630762, 'West Ham United': 1.7062146710685822, 'Wolverhampton Wanderers': 1.0880780371702332}

In [685]:
teamxg

{'Arsenal': 1.5906424644122763, 'Aston Villa': 1.0367192715181872, 'Brentford': 1.555792784872493, 'Brighton and Hove Albion': 1.5569590776464555, 'Burnley': 1.0814349351129797, 'Chelsea': 1.7425590162311395, 'Crystal Palace': 1.4103558308519641, 'Everton': 1.8145323987113366, 'Leeds United': 1.3371790070105611, 'Leicester City': 1.7747221005073104, 'Liverpool': 2.8172383179865466, 'Manchester City': 1.9319486526195437, 'Manchester United': 1.7187607603977826, 'Newcastle United': 1.5401989899293334, 'Norwich City': 1.2922233822183689, 'Southampton': 1.3888341502401684, 'Tottenham Hotspur': 1.4545498592331567, 'Watford': 1.5077940601630762, 'West Ham United': 1.7062146710685822, 'Wolverhampton Wanderers': 1.0880780371702332}

In [684]:
teamsxg[['team','xG_90']].groupby('team').mean()

,xG_90
team,
Arsenal,1.612320
Aston Villa,1.035177
Brentford,1.241835
Brighton,1.216195
Burnley,1.037316
Chelsea,1.786379
Crystal Palace,1.317771
Everton,1.262535
Leeds,1.309762


In [662]:
teamxg

{'Arsenal': 1.9326560947938711, 'Aston Villa': 1.2110564887852289, 'Brentford': 1.8088957866809303, 'Brighton and Hove Albion': 1.9707211581004582, 'Burnley': 1.6905206228762535, 'Chelsea': 2.0409049618581347, 'Crystal Palace': 2.2543900586060923, 'Everton': 2.424933990964974, 'Leeds United': 1.963822627337791, 'Leicester City': 2.4999242138348485, 'Liverpool': 2.991575535253588, 'Manchester City': 2.26985659702712, 'Manchester United': 2.318301238201326, 'Newcastle United': 1.8735885104449186, 'Norwich City': 1.5637578094801774, 'Southampton': 1.5631713675072105, 'Tottenham Hotspur': 2.2465732750738803, 'Watford': 2.1629592480232143, 'West Ham United': 2.0102594189725576, 'Wolverhampton Wanderers': 1.5114608275964805}

In [169]:
def poisspdf(x, mu):
    return mu**x * math.e**-mu / math.factorial(x)

In [273]:
poissl = []
for xg in teamxg.values():
    row = []
    for i in range(0,7):
        row.append(poisspdf(i, xg))
    poissl.append(row)


In [274]:
pd.DataFrame(poissl, index = teamxg.keys())

,0,1,2,3,4,5,6
Arsenal,0.264680,0.351821,0.233827,0.103604,0.034428,0.009153,0.002028
Aston Villa,0.484872,0.350984,0.127034,0.030652,0.005547,0.000803,0.000097
Brentford,0.301027,0.361399,0.216940,0.086816,0.026057,0.006257,0.001252
Brighton and Hove Albion,0.317469,0.364256,0.208969,0.079922,0.022925,0.005261,0.001006
Burnley,0.591883,0.310411,0.081397,0.014229,0.001866,0.000196,0.000017
Chelsea,0.229922,0.337989,0.248424,0.121729,0.044736,0.013153,0.003222
Crystal Palace,0.367797,0.367879,0.183981,0.061341,0.015339,0.003068,0.000512
Everton,0.194474,0.318443,0.260718,0.142305,0.058254,0.019078,0.005207
Leeds United,0.417674,0.364652,0.159181,0.046324,0.010111,0.001765,0.000257
Leicester City,0.306189,0.362391,0.214454,0.084606,0.025034,0.005926,0.001169
